react agent

In [109]:
from dotenv import load_dotenv

load_dotenv()

True

In [110]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
# 현재 멀티모달 제공하는 모델은 3가지지

import base64
import os
from openai import OpenAI

In [111]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [112]:
from pydantic import BaseModel, Field

class ImageDescription(BaseModel):
    category : str = Field(description = "이미지의 카테고리. 4가지 카테고리 : ['figure', 'chart', 'table', 'equation']")
    description : str = Field(description = "이미지의 설명")

In [113]:
def encode_image(image_path : str):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [1]:


image_path = "C:\\test_test\\response_rate_of_image_description\\data\\equation.png"




In [115]:
base64_image = encode_image(image_path)

llm = ChatOpenAI(model = "gpt-4.1" , temperature = 0)

In [116]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

from glom import glom

In [117]:
@tool
def get_image_description(image_path : str):
    """
    이미지 경로를 통해서 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하는 툴
    """
    base64_image = encode_image(image_path)

    ext  = os.path.splitext(image_path)[1]

    client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

    message_list = [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": "주어진 이미지를 분석하세요. 이미지 종류('figure', 'chart', 'table', 'equation')와 설명을 반환하세요. 표의 경우 담겨있는 정보를 생략하지 말고 전부 일목요연하게 제공하세요. 설명의 경우 마크다운으로 작성하세요요"
                },
                {
                    "type": "input_image",
                    "image_url": f"data:image/{ext};base64,{base64_image}",
                    "detail": "high"
                }
            ]
        }
    ]

    response = client.responses.parse(
        model = "gpt-4.1",
        input = message_list,
        text_format=ImageDescription
    )

    re_output = response.output_parsed

    category = glom(re_output, 'category', default = None)
    description = glom(re_output, 'description', default = None)

    return category, description


In [118]:
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
import requests
import json

In [119]:
@tool
async def upstage_parser(image_path : str):
    """
    이미지 경로를 통해서 이미지 종류('chart', 'table', 'equation')인 이미지를 markdown 형태로 반환하는 툴
    """

    headers = {"Authorization" : f"Bearer {UPSTAGE_API_KEY}"}

    files = {"document" : open(image_path, 'rb')}

    config = {
        
    }

    response = requests.post(
        "https://api.upstage.ai/v1/document-ai/document-parse",
        headers = headers,
        data = config,
        files = files
    )

    if response.status_code == 200:
        response_json = response.json()

        markdown_content = glom(response_json, 'content.markdown', default = None)

        return markdown_content

    else:
        return ''




In [120]:
from langchain_core.messages import SystemMessage

system_message = SystemMessage(
    content = f"""
    image_path {image_path}

    당신은 제공받은 이미지 경로를 이용하여 이미지 카테고리('figure', 'chart', 'table', 'equation')과 설명을 이용하여 이미지를 분석하는 봇입니다.

    get_image_description 툴을 이용하여 이미지 카테고리와 이미지 설명을 반환받으세요

    제공받은 설명을 완벽한 마크다운 형태로 리포맷팅 하세요.
    """
)

tools = [get_image_description]

llm = ChatOpenAI(model = "gpt-4.1", temperature = 0)

In [121]:
agent = create_react_agent(
    llm,
    tools,
    prompt = system_message
)

In [122]:
from langchain_core.runnables import RunnableConfig

inputs = {
    "messages" : [
        ("user", "주어진 이미지를 분석하세요")
    ]
}

config = RunnableConfig(
    recursion_limit=10
)



In [124]:
from langchain_teddynote.messages import stream_graph

from time import time

start_time = time()

stream_graph(agent, inputs, config)

end_time = time()

print(f"Time taken: {end_time - start_time} seconds")


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["figure", "이 이미지는 활짝 웃으며 춤을 추는 하마의 귀여운 캐릭터 일러스트입니다. 하마는 한 발로 서서 팔과 다리를 활짝 벌린 채, 매우 즐거운 표정을 하고 있습니다. 배경은 부드러운 색감의 자연 풍경으로 따뜻한 분위기를 연출합니다. 이 이미지는 유쾌하고 생동감 넘치는 분위기를 전달합니다."]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
## 이미지 분석 결과

- **카테고리:** Figure

- **설명:**  
  이 이미지는 활짝 웃으며 춤을 추는 하마의 귀여운 캐릭터 일러스트입니다. 하마는 한 발로 서서 팔과 다리를 활짝 벌린 채, 매우 즐거운 표정을 하고 있습니다. 배경은 부드러운 색감의 자연 풍경으로 따뜻한 분위기를 연출합니다. 이 이미지는 유쾌하고 생동감 넘치는 분위기를 전달합니다.Time taken: 11.24609637260437 seconds
